# Cleaning Data in Python
It's commonly said that data scientists spend 80% of their time cleaning and manipulating data and only 20% of their time analyzing it. The time spent cleaning is vital since analyzing dirty data can lead you to draw inaccurate conclusions.

Data cleaning is an essential task in data science. Without properly cleaned data, the results of any data analysis or machine learning model could be inaccurate. In this course, you will learn how to identify, diagnose, and treat a variety of data cleaning problems in Python, ranging from simple to advanced. You will deal with improper data types, check that your data is in the correct range, handle missing data, perform record linkage, and more!

## Chapter 1: Common data problems

In this chapter, I will learn how to overcome some of the most common dirty data problems: convert data types, apply range constraints to remove future data points, and remove duplicated data points to avoid double-counting.

In [3]:
# Import the required packages
import pandas as pd

In [4]:
# Load the data from the Ride Sharing CSV
ride_sharing = pd.read_csv("./data/ride_sharing_new.csv")

In [5]:
# Take a look at the first few rows
ride_sharing.head()

,Unnamed: 0,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender
0,0,12 minutes,81,Berry St at 4th St,323,Broadway at Kearny,5480,2,1959,Male
1,1,24 minutes,3,Powell St BART Station (Market St at 4th St),118,Eureka Valley Recreation Center,5193,2,1965,Male
2,2,8 minutes,67,San Francisco Caltrain Station 2 (Townsend St...,23,The Embarcadero at Steuart St,3652,3,1993,Male
3,3,4 minutes,16,Steuart St at Market St,28,The Embarcadero at Bryant St,1883,1,1979,Male
4,4,11 minutes,22,Howard St at Beale St,350,8th St at Brannan St,4626,2,1994,Male


In [6]:
# Take a look at the info of this dataframe
ride_sharing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25760 entries, 0 to 25759
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       25760 non-null  int64 
 1   duration         25760 non-null  object
 2   station_A_id     25760 non-null  int64 
 3   station_A_name   25760 non-null  object
 4   station_B_id     25760 non-null  int64 
 5   station_B_name   25760 non-null  object
 6   bike_id          25760 non-null  int64 
 7   user_type        25760 non-null  int64 
 8   user_birth_year  25760 non-null  int64 
 9   user_gender      25760 non-null  object
dtypes: int64(6), object(4)
memory usage: 2.0+ MB


We can see some data type issues here. The `duration` column type is `object`, which is how `pandas` stores strings, also the `user_gender` column is stored as `object`, while it should be `category` to facilitate analysis. 

Also, the column `user_type` is `int`, while it should be `category`.
The `user_type`` column contains information on whether a user is taking a free ride and takes on the following values:
- 1 for free riders.
- 2 for pay per ride.
- 3 for monthly subscribers.

In [7]:
# Print summary statistics of user_type column
print(ride_sharing['user_type'].describe())

count    25760.000000
mean         2.008385
std          0.704541
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: user_type, dtype: float64


We can see that `pandas` treats the datapoints numerically, as amounts rather than categories. We can't work with that.

The `user_type` column has a finite set of possible values that represent groupings of data, so it should be converted to `category`.

In [8]:
# Convert user_type from integer to category
ride_sharing['user_type_cat'] = ride_sharing['user_type'].astype("category")

In [9]:
# Write an assert statement confirming the change
assert ride_sharing['user_type_cat'].dtype == 'category'

### Exercise (Data Type Constraints): Summing strings and concatenating numbers
In this exercise, you'll be converting the string column `duration` to the type `int`. Before that however, you will need to make sure to strip `"minutes"` from the column in order to make sure `pandas` reads it as numerical.

In [10]:
ride_sharing.head()

,Unnamed: 0,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender,user_type_cat
0,0,12 minutes,81,Berry St at 4th St,323,Broadway at Kearny,5480,2,1959,Male,2
1,1,24 minutes,3,Powell St BART Station (Market St at 4th St),118,Eureka Valley Recreation Center,5193,2,1965,Male,2
2,2,8 minutes,67,San Francisco Caltrain Station 2 (Townsend St...,23,The Embarcadero at Steuart St,3652,3,1993,Male,3
3,3,4 minutes,16,Steuart St at Market St,28,The Embarcadero at Bryant St,1883,1,1979,Male,1
4,4,11 minutes,22,Howard St at Beale St,350,8th St at Brannan St,4626,2,1994,Male,2


In [11]:
"11 minutes".strip(" minutes")

'11'

In [12]:
# Strip duration of minutes
ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip("minutes")

# Convert duration to integer
ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype("int")

# Write an assert statement making sure of conversion
assert ride_sharing['duration_time'].dtype == 'int'

# Print formed columns and calculate average ride duration 
print(ride_sharing[['duration','duration_trim','duration_time']])
print("The average ride duration is", ride_sharing["duration_time"].mean(), "minutes.")

         duration duration_trim  duration_time
0      12 minutes           12              12
1      24 minutes           24              24
2       8 minutes            8               8
3       4 minutes            4               4
4      11 minutes           11              11
...           ...           ...            ...
25755  11 minutes           11              11
25756  10 minutes           10              10
25757  14 minutes           14              14
25758  14 minutes           14              14
25759  29 minutes           29              29

[25760 rows x 3 columns]
The average ride duration is 11.389052795031056 minutes.


### Exercise (Data Range Constraints): Tire size constraints
In this lesson, you're going to build on top of the work you've been doing with the `ride_sharing` DataFrame. You'll be working with the `tire_sizes` column which contains data on each bike's tire size.

Bicycle tire sizes could be either 26″, 27″ or 29″ and are here correctly stored as a `categorical` value. In an effort to cut maintenance costs, the ride sharing provider decided to set the maximum tire size to be 27″.

In this exercise, you will make sure the `tire_sizes` column has the correct range by first converting it to an `integer`, then setting and testing the new upper limit of 27″ for tire sizes.

In [13]:
# Simulating the exercise dataframe by adding the 'tire_sizes' column with the specified values
# but with some outliers above 27
import random

ride_sharing["tire_sizes"] = [random.randint(26,31) for _ in range(25760)]
ride_sharing["tire_sizes"] = ride_sharing["tire_sizes"].astype("category")
ride_sharing["tire_sizes"].value_counts()

tire_sizes
29    4420
28    4298
27    4290
26    4275
30    4253
31    4224
Name: count, dtype: int64

In [14]:
# Convert tire_sizes to integer
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')

# Set all values above 27 to 27
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, "tire_sizes"] = 27

# Reconvert tire_sizes back to categorical
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')

# Print tire size description
print(ride_sharing['tire_sizes'].describe())

count     25760
unique        2
top          27
freq      21485
Name: tire_sizes, dtype: int64


### Exercise (Data 'Date' Range Constraints): Back to the future
A new update to the data pipeline feeding into the `ride_sharing` DataFrame has been updated to register each ride's date. This information is stored in the `ride_date` column of the type object, which represents strings in `pandas`.

A bug was discovered which was relaying rides taken today as taken next year. To fix this, you will find all instances of the `ride_date` column that occur anytime in the future, and set the maximum possible value of this column to today's date. Before doing so, you would need to convert `ride_date` to a `datetime` object.

In [15]:
# Simulating the exercise dataframe by adding the 'ride_date' column with the specified values
# but with some outlier dates (in the future)
from datetime import datetime, timedelta
import random

# Function to generate a random date within a specified range
def random_date(start_date, end_date):
    time_delta = end_date - start_date
    random_days = random.randint(0, time_delta.days)
    random_date = start_date + timedelta(days=random_days)
    return random_date

# Define the range of dates
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 12, 31)

# Create the required column to simulate the exercise
ride_sharing["ride_date"] = [random_date(start_date, end_date) for _ in range(25760)]
# Make it of the type 'object'
ride_sharing["ride_date"] = ride_sharing["ride_date"].astype("object")

In [16]:
import datetime as dt

# Convert ride_date to date
ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date

# Save today's date
today = dt.date.today()

# Set all in the future to today's date
ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today

# Print maximum of ride_dt column
print(ride_sharing['ride_dt'].max())

2023-11-10


### Notes on Data Uniqueness Constraints: 
**To find duplicate rows**, you can use the `pd.df.duplicated()` method.
Specify its arguments as such:
- `subset`: list of column names to be checked for duplication.
- `keep`: whether to keep the first('`first`'), last('`last`'), or all('`false`') duplicate values in the result.
**To eliminate duplicate**, you can use the `pd.df.drop_duplicates()` method with the same arguments.

**Note there are two kinds of duplicates: partial and complete.**

### Exercise: Finding duplicates

A new update to the data pipeline feeding into `ride_sharing` has added the `ride_id` column, which represents a unique identifier for each ride.

The update however coincided with radically shorter average ride duration times and irregular user birth dates set in the future. Most importantly, the number of rides taken has increased by 20% overnight, leading you to think there might be both complete and incomplete duplicates in the `ride_sharing` DataFrame.

In [17]:
# Simulating the exercise dataframe by adding a column 'ride_id' with random numbers that are bound
# to have some duplicate values
ride_sharing["ride_id"] = [random.randint(1,26_000) for _ in range(25760)]

# Check for duplicates
ride_sharing["ride_id"].value_counts().sort_values()

ride_id
23543    1
19511    1
9721     1
9912     1
3577     1
        ..
21506    6
15001    6
17078    6
1744     6
9181     6
Name: count, Length: 16332, dtype: int64

In [18]:
# Find duplicates
duplicates = ride_sharing.duplicated(subset=["ride_id"], keep=False)

# Sort your duplicated rides
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print relevant columns of duplicated_rides
duplicated_rides[['ride_id','duration','user_birth_year']]

,ride_id,duration,user_birth_year
3252,8,5 minutes,1985
5787,8,23 minutes,1983
10052,10,7 minutes,1983
4019,10,9 minutes,1989
25356,17,9 minutes,1979
...,...,...,...
23407,25993,8 minutes,1969
1299,25994,13 minutes,1982
161,25994,7 minutes,1990
13037,25999,18 minutes,1981


Notice that all our duplicate rows in this case are *incomplete duplicates.*

### Exercise: Treating duplicates
In the last exercise, you were able to verify that the new update feeding into `ride_sharing` contains a bug generating incomplete duplicated rows for some values of the `ride_id` column, with occasional discrepant values for the `user_birth_year` and `duration` columns.

In this exercise, you will be treating those duplicated rows by first dropping complete duplicates (if any), and then merging the incomplete duplicate rows into one while keeping the average `duration`, and the minimum `user_birth_year` for each set of incomplete duplicate rows.

In [19]:
# To simulate the exercise dataframe
ride_sharing["duration"] = ride_sharing["duration"].str.strip("minutes").astype("int")

In [20]:
# Drop complete duplicates from ride_sharing (if any)
ride_dup = ride_sharing.drop_duplicates()

# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': "min", 'duration': "mean"}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

## Chapter 2: Text and categorical data problems

Categorical and text data can often be some of the messiest parts of a dataset due to their unstructured nature. In this chapter, you’ll learn how to fix whitespace and capitalization inconsistencies in category labels, collapse multiple categories into one, and reformat strings for consistency.

### Notes on Membership constraints:
We can have inconsistencies in categorical data for a variety of reasons such as data entry errors or parsing errors.
Treatments: dropping data, remapping categories, inferring categories.

It's always good practice to keep a log of all possible values of our categorical data, as it will make dealing with inconsistencies much easier.

In [21]:
# Remember from the set theory
set_A = {1,2,3,4}
set_B = {1,3,5,7,9}

# A left anti-join
set_A.difference(set_B)

{2, 4}

### Exercise: Finding consistency
In this exercise and throughout this chapter, you'll be working with the `airlines` DataFrame which contains survey responses on the San Francisco Airport from airline customers.

The DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding `cleanliness`, `safety`, and `satisfaction`. Another DataFrame named `categories` was created, containing all correct possible values for the survey columns.

In this exercise, you will use both of these DataFrames to find survey answers with inconsistent values, and drop them, effectively performing an outer and inner join on both these DataFrames as seen in the video exercise.

In [22]:
# Creating the categories dataframe to simulate the exercise
cleanliness = ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty']
safety = ['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe']
satisfaction = ['Very satisfied',
 'Neutral',
 'Somewhat satisfied',
 'Somewhat unsatisfied',
 'Very unsatisfied']

categories = pd.DataFrame({"cleanliness": cleanliness, "safety": safety, "satisfaction": satisfaction})
categories

,cleanliness,safety,satisfaction
0,Clean,Neutral,Very satisfied
1,Average,Very safe,Neutral
2,Somewhat clean,Somewhat safe,Somewhat satisfied
3,Somewhat dirty,Very unsafe,Somewhat unsatisfied
4,Dirty,Somewhat unsafe,Very unsatisfied


In [23]:
# Solving the exercise (Part 1 out of 3)
# Loading the dataframe
airlines = pd.read_csv("./data/airlines_final.csv")

# Print categories DataFrame
print(categories)

# Print unique values of survey columns in airlines
print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ', airlines['safety'].unique(), "\n")
print('Satisfaction: ', airlines['satisfaction'].unique(), "\n")

      cleanliness           safety          satisfaction
0           Clean          Neutral        Very satisfied
1         Average        Very safe               Neutral
2  Somewhat clean    Somewhat safe    Somewhat satisfied
3  Somewhat dirty      Very unsafe  Somewhat unsatisfied
4           Dirty  Somewhat unsafe      Very unsatisfied
Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty' 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'
 'Very unsatisfied'] 



In [24]:
# Find the cleanliness category in airlines not in categories
set(airlines["satisfaction"]).difference(categories["satisfaction"])

{'Somewhat satsified'}

In [25]:
# Find the cleanliness category in airlines not in categories
cat_clean = set(airlines["satisfaction"]).difference(categories["satisfaction"])

# Find rows with that category
cat_clean_rows = airlines['satisfaction'].isin(cat_clean)

# Print rows with inconsistent category
print(airlines[cat_clean_rows])

# Print rows with consistent categories only
print(airlines[~cat_clean_rows])

      Unnamed: 0    id        day         airline   destination  \
3              3  1157    Tuesday       SOUTHWEST   LOS ANGELES   
4              4  2992  Wednesday        AMERICAN         MIAMI   
6              6  2578   Saturday         JETBLUE    LONG BEACH   
8              9   919     Friday      AIR CANADA       TORONTO   
10            11  1129    Tuesday       SOUTHWEST     SAN DIEGO   
...          ...   ...        ...             ...           ...   
2468        2800  1942    Tuesday          UNITED        BOSTON   
2469        2801  2130   Thursday  CATHAY PACIFIC     HONG KONG   
2471        2803  2888  Wednesday          UNITED        AUSTIN   
2472        2804  1475    Tuesday          ALASKA  NEW YORK-JFK   
2476        2808  2162   Saturday   CHINA EASTERN       QINGDAO   

        dest_region  dest_size boarding_area   dept_time  wait_min  \
3           West US        Hub   Gates 20-39  2018-12-31     190.0   
4           East US        Hub   Gates 50-59  2018-12-3

### Exercise: Inconsistent categories

In this exercise, you will examine two categorical columns from this DataFrame, `dest_region` and `dest_size` respectively, assess how to address them and make sure that they are cleaned and ready for analysis.

In [27]:
# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     ']


Note: We can see there are some inconsistenices in the values of the `dest_region` columns due to capitalization. Inconsistencies also appear in the `dest_size` column due to leading and trailing spaces.

In [28]:
# Lower dest_region column and then replace "eur" with "europe"
airlines['dest_region'] = airlines['dest_region'].str.lower()
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

In [29]:
# Remove white spaces from `dest_size`
airlines['dest_size'] = airlines['dest_size'].str.strip(" ")

# Verify changes have been effected
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']
['Hub' 'Small' 'Medium' 'Large']


Now all the values in these two columns have been properly treated to be consistent.

### Exercise: Remapping categories

To better understand survey respondents from airlines, you want to find out if there is a relationship between certain responses and the day of the week and wait time at the gate.

The `airlines` DataFrame contains the `day` and `wait_min` columns, which are categorical and numerical respectively. The `day` column contains the exact day a flight took place, and `wait_min` contains the amount of minutes it took travelers to wait at the gate. To make your analysis easier, you want to create two new categorical variables:
- `wait_type`: `'short'` for 0-60 min, `'medium'` for 60-180 and `long` for 180+
- `day_week`: `'weekday'` if day is in the weekday, `'weekend'` if day is in the weekend.


In [ ]:
import numpy as np

# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

You just created two new categorical variables, that when combined with other columns, could produce really interesting analysis. Don't forget, you can always use an `assert` statement to check your changes passed.

### Exercise: Removing titles and taking names

While collecting survey respondent metadata in the airlines DataFrame, the full name of respondents was saved in the `full_name` column. However upon closer inspection, you found that a lot of the different names are prefixed by honorifics such as "Dr.", "Mr.", "Ms." and "Miss".

Your ultimate objective is to create two new columns named `first_name` and `last_name`, containing the first and last names of respondents respectively. Before doing so however, you need to remove honorifics.

In [46]:
# Adding the full_name column to simulate the original exercise
import numpy as np

RAW_FULL_NAMES = ['Melodie Stuart', 'Dominic Shannon', 'Quintessa Tillman',
       'Dr. Christine Nicholson', 'Regina Clements', 'Colleen Harding',
       'Kaitlin Cochran', 'Molly Norton', 'Richard Lott',
       'Matthew Nguyen', 'Dr. Laith Decker', 'Holly Austin', 'Jaden Gray',
       'Germaine Hurley', 'Kyle Gay', 'Zachery Diaz', 'Carolyn Hartman',
       'Miss Alana Grant', 'Idola Acosta', 'Dara English',
       'Miss Aurora Flores', 'Henry Sloan', 'Jared Chase',
       'Xavier Castro', 'Holmes Fowler', 'Lucy Noel', 'Kerry Tucker',
       'Garrison Barrett', 'Stephanie Cannon', 'Dr. Charlotte Savage',
       'Lane Clements', 'Aimee Whitfield', 'Martena Neal',
       'Xandra Hartman', 'Meredith Gutierrez', 'Mr. Kermit Deleon',
       'Derek Terrell', 'Shaeleigh Mccarthy', 'Burke Leon',
       'Mr. Clinton Holmes', 'Whoopi Tillman', 'Hamilton Gardner',
       'Graiden Bridges', 'Sheila Robinson', 'Cameron Barlow',
       'Kasimir Irwin', 'Ms. Lilah Chen', 'Judith Price', 'Dane Barker',
       'Micah Bullock', 'Leonard Stevens', 'Ms. Beverly Hampton',
       'Devin Morrison', 'Mr. Jordan Cooke', 'Miss Ann Hale',
       'Graiden Riddle', 'Julian Stanley', 'Christine Carter',
       'Hasad Valentine', 'Bevis Mcdowell', 'Alec Davis',
       'Dr. Daniel Hood', 'Ms. Britanney Schmidt', 'Wanda Jackson',
       'Quyn Henderson', 'Hammett Duncan', 'Duncan Stark', 'Jin Shannon',
       'Fulton Meadows', 'Dr. Malik Hanson', 'Laith Espinoza',
       'Dr. Jared Holman', 'Julie Davidson', 'Dr. Jane Harrell',
       'Aphrodite Shannon', 'Jermaine Randall', 'Hammett Talley',
       'Sasha Riggs', 'Dr. Damian Wynn', 'Aidan Macias', 'Sawyer Hines',
       'Mr. Hector Caldwell', 'Abra Webb', 'Stone Price',
       'Cheyenne Stout', 'Lareina Wall', 'Dr. Ella Pena',
       'Quintessa Sherman', 'Ishmael Duffy', 'Ms. Willa Stuart',
       'Gareth Hunt', 'Stewart Jacobs', 'Ms. Amaya Pate',
       'Dr. Xavier Medina', 'Mr. Marvin Mcneil', 'Imogene Harris',
       'Abbot Hensley', 'Miss Fiona Velez', 'Rinah Stephenson',
       'Ms. Olivia Keith', 'Vielka Rosario', 'Lani Sawyer',
       'Clayton Sparks', 'Oprah Ingram', 'Acton Smith', 'Demetria Byrd',
       'Patience Galloway', 'Hoyt Alvarez', 'Dara Pennington',
       'Ebony Davidson', 'Brent Rosario', 'Melyssa Mayer', 'Regan Kelly',
       'Leah Barlow', 'Nathan Santos', 'Uta Mckee', 'Lawrence Gallegos',
       'Matthew Edwards', 'Xander Wilson', 'Kelly Pittman', 'Brynne Pugh',
       'Shea Collins', 'Hu Carver', 'Stacey Coleman', 'Kaye Mcgowan',
       'Vivien Cobb', 'Vaughan Harrison', 'Porter Hudson', 'Carl Conway',
       'Lyle Bradshaw', 'Hashim Walter', 'Branden Larson', 'Idola Ball',
       'Camilla White', 'Rafael Lowery', 'Victor Leon', 'Yasir Lynch',
       'Dr. Emerson Woodard', 'Dr. Astra Mcneil', 'Dr. Shafira Marks',
       'Mr. Dominic Smith', 'Talon Holder', 'Ivor Wise', 'Carolyn Clay',
       'Jerome Ruiz', 'Todd Chase', 'Gray Noel', 'Ann Sanchez',
       'Mr. Alec Heath', 'Heidi Terry', 'Alana Velasquez',
       'Mr. Jared York', 'Abbot Lester', 'Dr. Fulton Turner',
       'Dr. Maggie Cortez', 'Ramona Wade', 'Dr. Lynn Thomas',
       'Aquila Graham', 'Gareth Marks', 'Dolan Wolf', 'Julie Coffey',
       'Emerson Hatfield', 'Claire Rios', 'Christian Doyle',
       'Haley Oliver', 'Rigel Day', 'Clare Gould', 'Ms. Keiko Mcfarland',
       'Duncan Chandler', 'Penelope Stark', 'Kasper Shields',
       'Dr. Rose Fleming', 'Miss Petra Mitchell', 'Ms. Regan Lynch',
       'Keane Bennett', 'Nash Head', 'Ainsley Riley', 'Kirestin Newton',
       'Jakeem Hall', 'Reece Mitchell', 'Wanda Walls', 'Barry Mccray',
       'Dr. Zahir Hardin', 'Graiden Cox', 'Miss Lara Green', 'Felix Bell',
       'Mr. Addison Day', 'Tallulah Guzman', 'Jocelyn Guzman',
       'Ivory Miller', 'Mr. Eaton Vazquez', 'Silas Clemons',
       'Quinn Barry', 'Orson Pratt', 'Constance Morse',
       'Ms. Vanna Rivera', 'Miss Venus Lowe', 'Amethyst Nieves',
       'Miss Vivian Foreman', 'Miss Wendy Griffin']

names_array = np.full(2477, np.nan, dtype='object')
names_array[:200] = RAW_FULL_NAMES
airlines["full_name"] = names_array 

In [70]:
# for the purposes of this exercise alone let's dump all the NAN values for names
airlines_with_names = airlines.dropna()
airlines_with_names.shape

(200, 14)

In [71]:
# Replace "Dr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")

# Replace "Mr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")

# Replace "Miss" with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Miss","")


# Replace "Ms." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")

# Assert that full_name has no honorifics
assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

By normalizing full names this way, you can now easily split them into first names and last names!

### Exercise: Keeping it descriptive

To further understand travelers' experiences in the San Francisco Airport, the quality assurance department sent out a qualitative questionnaire to all travelers who gave the airport the worst score on all possible categories. The objective behind this questionnaire is to identify common patterns in what travelers are saying about the airport.

Their response is stored in the `survey_response` column. Upon a closer look, you realized a few of the answers gave the shortest possible character amount without much substance. In this exercise, you will isolate the responses with a character count higher than 40, and make sure your new DataFrame contains responses with 40 characters or more using an `assert` statement.

In [72]:
# Simulating the original exercise

surveys_array = np.full(2477, np.nan, dtype='object')
RAW_SURVEYS = np.array(['It was terrible', "I didn't like the flight", 'I hate this ',
       'Not a fan', 'Bad', 'Horrible', 'Very poor', 'Unacceptable flight',
       'It was awful', 'My fllight was really unpleasant',
       'I am not a fan', 'I had a bad flight', 'It was very bad',
       'it was horrible', 'Terrible', 'It was substandard',
       'I did not enjoy the flight',
       'The airport personnell forgot to alert us of delayed flights, the bathrooms could have been cleaner',
       'The food in the airport was really really expensive - also no automatic escalators!',
       'One of the other travelers was really loud and talkative and was making a scene and no one did anything about it',
       "I don't remember answering the survey with these scores, my experience was great! ",
       'The airport personnel kept ignoring my requests for directions ',
       'The chair I sat in was extremely uncomfortable, I still have back pain to this day! ',
       'I wish you were more like other airports, the flights were really disorganized! ',
       'I was really unsatisfied with the wait times before the flight. It was unacceptable.',
       "The flight was okay, but I didn't really like the number of times I had to stop at security",
       'We were really slowed down by security measures, I missed my flight because of it! ',
       'There was a spill on the aisle next to the bathroom and it took hours to clean!',
       'I felt very unsatisfied by how long the flight took to take off.'],
      dtype=object)
surveys_array[:29] = RAW_SURVEYS

airlines["survey_response"] = surveys_array
airlines_with_surveys = airlines.dropna()
# Check
airlines_with_surveys.sample(3)

,Unnamed: 0,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,full_name,survey_response
4,4,2992,Wednesday,AMERICAN,MIAMI,east us,Hub,Gates 50-59,2018-12-31,559.0,Somewhat clean,Very safe,Somewhat satsified,Regina Clements,Bad
6,6,2578,Saturday,JETBLUE,LONG BEACH,west us,Small,Gates 1-12,2018-12-31,63.0,Clean,Very safe,Somewhat satsified,Kaitlin Cochran,Very poor
10,11,1129,Tuesday,SOUTHWEST,SAN DIEGO,west us,Medium,Gates 20-39,2018-12-31,540.0,Clean,Very safe,Somewhat satsified,Laith Decker,I am not a fan


In [74]:
# Store length of each row in survey_response column
resp_length = airlines["survey_response"].str.len()

# Find rows in airlines where resp_length > 40
airlines_survey = airlines[resp_length > 40]

# Assert minimum survey_response length is > 40
assert airlines_survey["survey_response"].str.len().min() > 40

# Print new survey_response column
print(airlines_survey['survey_response'])

17    The airport personnell forgot to alert us of d...
18    The food in the airport was really really expe...
19    One of the other travelers was really loud and...
20    I don't remember answering the survey with the...
21    The airport personnel kept ignoring my request...
22    The chair I sat in was extremely uncomfortable...
23    I wish you were more like other airports, the ...
24    I was really unsatisfied with the wait times b...
25    The flight was okay, but I didn't really like ...
26    We were really slowed down by security measure...
27    There was a spill on the aisle next to the bat...
28    I felt very unsatisfied by how long the flight...
Name: survey_response, dtype: object


These types of feedbacks are essential to improving any service. Coupled with some wordcount analysis, you can find common patterns across all survey responses in no time!

## Chapter 3: Advanced data problems

In this chapter, you’ll dive into more advanced data cleaning problems, such as ensuring that weights are all written in kilograms instead of pounds. You’ll also gain invaluable skills that will help you verify that values have been added correctly and that missing values don’t negatively impact your analyses.

## Chapter 4: Record linkage

Record linkage is a powerful technique used to merge multiple datasets together, used when values have typos or different spellings. In this chapter, you'll learn how to link records by calculating the similarity between strings—you’ll then use your new skills to join two restaurant review datasets into one clean master dataset.